## Demo

**Dataset**: Telco Customer Churn

Cada fila representa un cliente, cada columna contiene los atributos del cliente.

El conjunto de datos incluye información sobre:

- Clientes que se fueron en el último mes: Churn

- Servicios a los que cada cliente se ha suscrito: teléfono, líneas múltiples, Internet, seguridad en línea, respaldo en línea, protección de dispositivos, soporte técnico y transmisión de TV y películas.

- Información de la cuenta del cliente: cuánto tiempo ha sido cliente, contrato, método de pago, facturación electrónica, cargos mensuales y cargos totales

- Información demográfica sobre los clientes: género, rango de edad y si tienen parejas y dependiente

Infomacion dataset en el siguiente enlace: [link](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)

In [1]:
!pip install -q awswrangler

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.44 requires botocore==1.29.44, but you have botocore 1.29.74 which is incompatible.
aiobotocore 2.0.1 requires botocore<1.22.9,>=1.22.8, but you have botocore 1.29.74 which is incompatible.


In [2]:
import awswrangler as wr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams["figure.figsize"] =8,4 # para ajustar el tamaño de imagen
import warnings
warnings.filterwarnings("ignore") #para evitar errores de operaciones matematicas
import random
import statistics
import math
#from pandas_profiling import ProfileReport

In [3]:
dataset = wr.s3.read_csv(path='s3://data-tecnicas-muestreo/datasets/Telco_customer_churn.csv')

In [4]:
display(dataset.head())
display(dataset.shape)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Premium Tech Support,Unlimited Data,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Customer Status,Churn Score.1,Churn Category
0,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,Yes,0.0,0,20.94,129.09,1,Churned,86,Competitor
1,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,Yes,0.0,0,18.24,169.89,2,Churned,67,Other
2,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,No,Yes,0.0,0,97.20,917.70,3,Churned,86,Other
3,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,0.0,0,136.92,3182.97,3,Churned,84,Other
4,0280-XJGEX,Male,0,No,No,49,Yes,Yes,Fiber optic,No,...,No,Yes,0.0,0,2172.17,7208.47,1,Churned,89,Competitor


(7043, 52)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 52 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   customerID                         7043 non-null   object 
 1   gender                             7043 non-null   object 
 2   SeniorCitizen                      7043 non-null   int64  
 3   Partner                            7043 non-null   object 
 4   Dependents                         7043 non-null   object 
 5   tenure                             7043 non-null   int64  
 6   PhoneService                       7043 non-null   object 
 7   MultipleLines                      7043 non-null   object 
 8   InternetService                    7043 non-null   object 
 9   OnlineSecurity                     7043 non-null   object 
 10  OnlineBackup                       7043 non-null   object 
 11  DeviceProtection                   7043 non-null   objec

### Ubicamos la variable de interes para la estimacion puntual (media aritmetica)

In [6]:
dataset["tenure"].describe()

count    7043.000000
mean       32.371149
std        24.559481
min         0.000000
25%         9.000000
50%        29.000000
75%        55.000000
max        72.000000
Name: tenure, dtype: float64

### Ubicamos la variable de interes para la estimacion proporcional (variable dicotomica)

In [7]:
dataset['churn_rate'].value_counts()

0    5174
1    1869
Name: churn_rate, dtype: int64

### Estimacion puntual 

#### Tenure: Número de meses que el cliente ha permanecido en la empresa

In [8]:
target = dataset['tenure']

In [9]:

def mas_puntual(target, LEE):
    
     random.seed(123) # fijar semilla valores aleatorios
    
     mean_poblacional = target.mean()   # promedio poblacional (variable de interes)
     N = len(target) 
     D =  pow(LEE, 2)/4    # limite margenes error estimacion poblacional
     var = statistics.variance(target)   # Varianza Poblacional
     n =  math.floor((N*var)/((N-1)*D + var)) # seleccion de muestra aleatoria
    
    
     index_sample = random.sample (list(target.index), n)  # muestreo aleatorio simple
     target_sample = target[index_sample]  # seleccion muestra por index
    
     mean_est = target_sample.mean()  # promedio muestral estimado (variable de interes)
    
     var_mean_est = (np.var(target_sample)/n)*(1-(n/N))  # varianza muestral estimada (variable de interes)
     error_est =  2*math.sqrt(var_mean_est)  # error de estimacion muestra estimada (variable de interes)

     LI =  mean_est - error_est  # limite superior estimacion muestral
     LS =  mean_est + error_est  # limite inferior estimacion muestral
    
     data_filtrada = dataset.iloc[target_sample.index, :]  # dataset muestral
    
     print("El intervalo de confianza de la proporcion esta entre", "[",LI, LS,"]")
     print("tamaño total poblacional:" , len(dataset) )
     print("tamaño de muestra estimada:" , len(data_filtrada) )
     print("promedio poblacional estimado:" , mean_poblacional )
     print("promedio muestral estimado:" , mean_est )
     print("error de estimacion:" , error_est )
    
     return(data_filtrada)

In [10]:
df_mas_puntual = mas_puntual(target, 1.1)

El intervalo de confianza de la proporcion esta entre [ 31.252716880185897 33.4486988212298 ]
tamaño total poblacional: 7043
tamaño de muestra estimada: 1554
promedio poblacional estimado: 32.37114865824223
promedio muestral estimado: 32.35070785070785
error de estimacion: 1.097990970521951


In [11]:
df_mas_puntual.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Premium Tech Support,Unlimited Data,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Customer Status,Churn Score.1,Churn Category
428,4248-QPAVC,Female,1,Yes,No,17,Yes,Yes,Fiber optic,Yes,...,No,Yes,0.0,0,625.94,2089.39,1,Churned,90,Competitor
2192,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,Yes,0.0,0,25.18,144.68,4,Joined,25,0
714,0415-MOSGF,Female,0,No,No,1,Yes,No,DSL,No,...,No,Yes,0.0,0,26.53,70.93,2,Churned,76,Attitude
6299,0020-JDNXP,Female,0,Yes,Yes,34,No,No phone service,DSL,Yes,...,Yes,Yes,0.0,0,0.00,1993.20,4,Stayed,37,0
3336,7710-JSYOA,Female,0,Yes,Yes,69,Yes,Yes,Fiber optic,No,...,Yes,Yes,0.0,0,1968.57,8366.62,5,Stayed,45,0


### Interpretacion:



En resumen, la muestra nos indica que la cantidad de tiempo promedio que los clientes han permanecido en la empresa es de 32 (32.35) meses, osea alrededore de 2 años y ocho meses. Haciendo un comparativo la estimacion muestral se asemeja mucho a la metrica poblacional que fue de 32.37 meses.

Es necesario aproximadamente 1554 observaciones (clientes distintos) para estimar la cantidad promedio de meses que el cliente ha permanecido en la empresa, teniedo en cuenta un error de 1.09.

Entre mas pequeño sea el LEE (limite de error de estimacion), el tamaño de muestra aumentara.

Ademas, la estimacion promedio muestral se encuentra dentro de intervalo de confianza estimado,entonces podemos asegurar que 95 de cada 100 observaciones (clientes) contiene la cantidad promedio verdadera de meses que el cliente ha permanecido en la empresa (poblacional) 


### Guardamos el dataset muestra en la ruta S3

In [12]:
wr.s3.to_csv(df = df_mas_puntual, path = 's3://data-tecnicas-muestreo/datasets/Telco_customer_churn_mas_puntual.csv', index=False)

{'paths': ['s3://data-tecnicas-muestreo/datasets/Telco_customer_churn_mas_puntual.csv'],
 'partitions_values': {}}

### Estimacion proporcional

#### Si el cliente abandona o no (Sí = 1 o No = 0)

In [36]:
target = dataset['churn_rate']

In [37]:

def mas_proporcional(target, LEE):
    
     random.seed(123) # fijar semilla valores aleatorios
        
     p = target.mean()   # proporcion casos positivos (valores 1) poblacional
     q= 1 - p    # complemento proporcion CP poblacional
     N = len(target)   # tamaño poblacional
     D =  pow(LEE, 2)/4    # limite margenes error estimacion poblacional
     n =  math.floor((N*p*q)/((N-1)*D + p*q))  # tamaño muestral estimado
     
     index_sample = random.sample (list(target.index), n)  # muestreo aleatorio simple
     target_sample = target[index_sample]  # seleccion muestra por index
    
     p_est = target_sample.mean()  # proporcion casos positivos estimados muestra
     q_est =  1 - p_est   # complemento proporcion casos positivos estimados muestra
     var_p_est = ((p_est*q_est)/(n-1))/(1 - n/N) # varianza proporcion CP estimada muestra
     error_est =  2*math.sqrt(var_p_est)  # error de estimacion proporcion CP

     LI =  p_est - error_est  # limite superior estimado estimacion poblacional
     LS =  p_est + error_est  # limite inferior estimado estimacion poblacional
     
     data_filtrada = dataset.iloc[target_sample.index, :]    
        
     print("El intervalo de confianza de la proporcion esta entre", "[",LI, LS,"]")
     print("tamaño total poblacional:" , len(dataset) )
     print("tamaño de muestra estimada:" , len(data_filtrada) )
     print("proporcion casos positivos poblacional:" , p )
     print("proporcion casos positivos estimados:" , p_est )
     print("error de estimacion:" , error_est )
    
     return(data_filtrada)


In [38]:
df_mas_proporcional = mas_proporcional(target, 0.020)

El intervalo de confianza de la proporcion esta entre [ 0.23968690619888802 0.2907649602058271 ]
tamaño total poblacional: 7043
tamaño de muestra estimada: 1527
proporcion casos positivos poblacional: 0.2653698707936959
proporcion casos positivos estimados: 0.26522593320235754
error de estimacion: 0.02553902700346953


In [39]:
df_mas_proporcional.churn_rate.value_counts()

0    1122
1     405
Name: churn_rate, dtype: int64

In [40]:
dataset.churn_rate.value_counts()

0    5174
1    1869
Name: churn_rate, dtype: int64

In [28]:
display(df_mas_proporcional.head())
print(df_mas_proporcional.shape)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Premium Tech Support,Unlimited Data,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Customer Status,Churn Score.1,Churn Category
263,4822-RVYBB,Male,1,No,No,8,Yes,Yes,Fiber optic,No,...,No,Yes,0.0,0,323.60,1143.00,1,Churned,90,Attitude
3916,6366-XIVKZ,Female,0,Yes,Yes,13,Yes,No,DSL,Yes,...,No,Yes,0.0,0,509.60,1326.40,3,Stayed,46,0
3615,4559-UWIHT,Male,0,Yes,No,14,Yes,Yes,Fiber optic,No,...,No,Yes,0.0,0,637.98,1822.98,5,Stayed,79,0
4600,4227-OJHAL,Female,0,Yes,Yes,68,Yes,Yes,DSL,Yes,...,No,Yes,0.0,0,1370.20,6533.20,4,Stayed,67,0
3909,8929-KSWIH,Male,0,No,No,25,Yes,No,Fiber optic,No,...,Yes,Yes,0.0,0,123.50,2637.00,4,Stayed,23,0


(1527, 52)


### Interpretacion:



La muestra proporcional nos indica que 26.39% de observaciones (clientes) decidio abandonar la empresa de telecomunicaciones, esta
propocional se asemeja mucho probabilidad real de abandono poblacional de 26.53%.

Es necesario aproximadamente 1527 observaciones (clientes distintos) para estimar la proporcional de clientes que abandonaron la empresa en el ultimo año, teniedo en cuenta un error de 0.025.

Entre mas pequeño sea el LEE (limite de error de estimacion), el tamaño de muestra aumentara.

Ademas, la estimacion proporcional muestral de clientes que abandonaron la empresa se encuentra dentro de intervalo de confianza estimado, entonces podemos asegurar que 95 de cada 100 observaciones de clientes que abandonaron la empresa en la muestra son verdaderamente clientes que abandonaron la empresa en la poblacion. 


In [17]:
wr.s3.to_csv(df = df_mas_proporcional, path = 's3://data-tecnicas-muestreo/datasets/Telco_customer_churn_mas_proporcional.csv', index=False)

{'paths': ['s3://data-tecnicas-muestreo/datasets/Telco_customer_churn_mas_proporcional.csv'],
 'partitions_values': {}}

# Demo utilizando training job

### Para la estimacion proporcional

### -`Ejecutamos 02muestreo_aleatorio_simple_pipeline`  

